In [ ]:
%matplotlib inline
import sys, os, time
# Select GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
# from tensorflow.keras.applications.mobilenet import preprocess_input, MobileNet
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50

import sys
sys.path.append("../")

In [ ]:
BASE_PATH = "/home/ailie/Repos/BBAttacks/data/"

In [ ]:
IMAGES_PATH = os.path.join(BASE_PATH, "x_val_0_10000.npy")
LABELS_PATH = os.path.join(BASE_PATH, "y_val.npy")

In [ ]:
x_val_raw = np.load(IMAGES_PATH) # loaded as RGB
# didn't upload imagenet data here as it is too large, but it is publicl available
# and the experiments are reproducible 
x_val = preprocess_input(x_val_raw.copy()) # converted to BGR

In [ ]:
y_val = np.load(LABELS_PATH)

In [ ]:
x_val2 = x_val[:1000].copy()
y_val2 = y_val[:1000].copy()
x_val_raw2 = x_val_raw.copy()

In [ ]:
del(x_val)
del(y_val)
del(x_val_raw)

In [ ]:
import gc
gc.collect()

In [ ]:
x_val = x_val2
y_val = y_val2
x_val_raw = x_val_raw2

In [ ]:
y_val_one_hot = to_categorical(y_val, 1000)

In [ ]:
keras_idx_to_name = {}
f = open("imagenet/data/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()

In [ ]:
def top_k_accuracy(y_true, y_pred, k=1):
    '''From: https://github.com/chainer/chainer/issues/606
    
    Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

In [ ]:
K.clear_session()
# model = MobileNet()
model = ResNet50(weights='imagenet')

In [ ]:
import random
random.seed(40)
sample_indices = random.choices(range(len(x_val)),k=100)

In [ ]:
# sample_indices = range(500)
y_val_one_hot_sample = y_val_one_hot[sample_indices]
x_val_sample = x_val[sample_indices]
x_val_raw_sample = x_val_raw[sample_indices]

In [ ]:
y_pred_sample = model.predict(x_val_sample, verbose=1)

In [ ]:
top_k_accuracy(y_val_one_hot_sample, y_pred_sample, k=1)

In [ ]:
top_k_accuracy(y_val_one_hot_sample, y_pred_sample, k=5)

In [ ]:
indices = []
agreements = np.argmax(y_pred_sample, axis=1) == np.argmax(y_val_one_hot_sample, axis=1)
for x in range(len(agreements)):
    if agreements[x]:
        indices.append(x)

In [ ]:
y_val_one_hot_sample = y_val_one_hot_sample[indices][:100]
x_val_sample = x_val_sample[indices][:100]
x_val_raw_sample = x_val_raw_sample[indices][:100]

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Dense, Flatten, BatchNormalization
import importlib
import tensorflow as tf

import sys
sys.path.append("attacks/")
sys.path.append("utils/")

import utils
from data_manager import load_data

# Black Box Attacks
import random_noise
import EvoStrategy
import SimbaWrapper

<h3>Random attack</h3>

<h4>Debugging the random attack</h4>

In [ ]:
agreements_cell_test

In [ ]:
np.argmax(y_pred_sample, axis=1) == np.argmax(y_val_one_hot_sample, axis=1)

In [ ]:
plt.imshow(saved_noisy_imgs_cell_test[0])
print("correct label:", y_val[0])
print("after:", np.argmax(model.predict(np.expand_dims(saved_noisy_imgs_cell_test[0],0)), axis=1)[0])
print("initially:", np.argmax(model.predict(np.expand_dims(x_val[0],0)), axis=1)[0])

<h3>Evolutionary attack</h3>

In [ ]:
x_val[0].min()

In [ ]:
index = 6
plt.imshow((x_val[index]-x_val[index].min()).astype(int))

In [ ]:
from tqdm import tqdm

In [ ]:
importlib.reload(EvoStrategy)
perturbed_images = 0
adv_evo_strategy = {}
for index in tqdm(range(0,50)):
#     print()
#     print(index)
    img = x_val_raw_sample[index]
#     print(img)
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=15, 
        one_step_perturbation_pixel_count=1,
        verbose=False,
        zero_one_scale=False,
        range_scale_int=True,
#         max_rand=int(x_val.max()),
#         min_rand=int(x_val.min())
        preprocess = preprocess_input
    )
    no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=1000)
    if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
        perturbed_images += 1

In [ ]:
# importlib.reload(EvoStrategy)
# # perturbed_images = 0
# # adv_evo_strategy = {}
# for index in range(50,100):
#     print()
#     print(index)
#     img = x_val_raw_sample[index]
# #     print(img)
#     label = np.argmax(y_val_one_hot_sample[index])
#     adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
#         model=model,
#         img=img,
#         label=label,
#         generation_size=20, 
#         one_step_perturbation_pixel_count=1,
#         verbose=True,
#         zero_one_scale=False,
#         range_scale_int=True,
# #         max_rand=int(x_val.max()),
# #         min_rand=int(x_val.min())
#         preprocess = preprocess_input
#     )
#     no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=1000)
#     if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
#         perturbed_images += 1

In [ ]:
for index in range(50):
    if not adv_evo_strategy[index].stop_criterion():
        print(index)

In [ ]:
count_q = 0
sum_q = 0
query_list = []
fair_indices = []
for index in range(50):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        query_list.append(adv_evo_strategy[index].queries)
        fair_indices.append(index)
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

In [ ]:
np.median(query_list)

In [ ]:
np.mean(query_list)

In [ ]:
plt.hist(query_list)

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))
plt.title("Query histogram for EvoBA(1, 15) on ImageNet", fontdict={"size":22})
plt.hist(query_list)
plt.show()

In [ ]:
import math
from tqdm import tqdm
l0_dists = []
for index_diff in tqdm(fair_indices):
    diff = np.abs(adv_evo_strategy[index_diff].get_best_candidate() - x_val_raw_sample[index_diff])
#     diff = np.reshape(diff, (32, 32, 3))
    diff = (diff!=0)
    l0_dist = np.sum(diff)
    l0_dists.append(l0_dist)
#     print("L2 distance:", math.sqrt(np.sum(np.reshape(diff, (-1))**2)))
#     plt.imshow(np.reshape(adv_evo_strategy[index_diff].get_best_candidate(), (28, 28)))
#     plt.show()
#     print("Prediction:", model.predict(np.array([adv_evo_strategy[index_diff].get_best_candidate()])))

In [ ]:
np.median(l0_dists)

In [ ]:
plt.hist(queries)

In [ ]:
np.median(l2_dists)/255

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))
plt.title("L0 histogram for EvoBA(1, 15) on ImageNet", fontdict={"size":22})
plt.hist(l0_dists)
plt.show()

In [ ]:
import math
l2_dists = []
for index_diff in tqdm(fair_indices):
    diff = np.abs(adv_evo_strategy[index_diff].get_best_candidate() - x_val_raw_sample[index_diff])
#     diff = np.reshape(diff, (32, 32, 3))
    l2_dist = math.sqrt(np.sum(np.reshape(diff, (-1))**2))
    l2_dists.append(l2_dist)
#     print("L2 distance:", math.sqrt(np.sum(np.reshape(diff, (-1))**2)))
#     plt.imshow(np.reshape(adv_evo_strategy[index_diff].get_best_candidate(), (28, 28)))
#     plt.show()
#     print("Prediction:", model.predict(np.array([adv_evo_strategy[index_diff].get_best_candidate()])))

In [ ]:
np.shape(x_val_sample[0])

In [ ]:
np.mean(l2_dists)/255

In [ ]:
np.mean(l2_dists)/(255 * 224 * 224 * 3)

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(20, 9))
fig.tight_layout()
plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.1, wspace=0.2)

columns = 5
rows = 2
img_curr = 0
for i in range(1, columns + 1):
    img_indx = fair_indices[imgss[img_curr]]
    initial = (keras_idx_to_name[adv_evo_strategy[img_indx].label])
    if len(initial) > 30:
        initial = initial[:21] + "\n" + initial[21:]
    fig.add_subplot(rows, columns, i)
    plt.title(f"Original, \n {initial}", fontdict={"size":18})
    img_start = adv_evo_strategy[img_indx].img.astype(int)
    plt.imshow(img_start)
    
    fig.add_subplot(rows, columns, i + 5)
    
    img_final = adv_evo_strategy[img_indx].get_best_candidate().astype(int)
    predss = model.predict(np.expand_dims(img_final, axis=0))[0]
    predicted = np.argmax(predss)
    diff = math.sqrt(np.sum((img_final - img_start) **2))
    l2_distance = int((diff/(255)) * 100)/100
    l0_distance = (img_start != img_final).sum()
    final = (keras_idx_to_name[predicted])
    plt.title(f"Perturbed, \n {final} L2 distance:{l2_distance}\n L0 distance:{l0_distance}", fontdict={"size":18})
    plt.imshow(img_final)
    img_curr += 1
plt.show()

In [ ]:
for index_diff in tqdm(np.array(fair_indices)[imgss]):
    print(index_diff)
    initial = (keras_idx_to_name[adv_evo_strategy[index_diff].label])
    img = adv_evo_strategy[index_diff].get_best_candidate().astype(int)
    plt.imshow(img.astype(int))
    predicted = (np.argmax(model.predict(np.expand_dims(img, axis=0))[0]))
    final = (keras_idx_to_name[predicted])
    plt.show()

In [ ]:
importlib.reload(EvoStrategy)
from tqdm import tqdm
perturbed_images_bf = 0
adv_evo_strategy_bf = {}
for index in tqdm(range(len(x_val_sample))):
    if index % 10 == 0:
        verbose = True
    else:
        verbose = False
    img = x_val_sample[index]
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy_bf[index] = EvoStrategy.AdversarialPerturbationBFStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=20, 
        one_step_perturbation_pixel_count=5,
        verbose=verbose
    )
    no_steps_bf = adv_evo_strategy_bf[index].run_adversarial_attack(steps=100)
    if adv_evo_strategy_bf[index].stop_criterion() and no_steps_bf > 0:
        perturbed_images_bf += 1
    adv_evo_strategy_bf[index].active_generation = []
    adv_evo_strategy_bf[index].fitness_scores = []
    gc.collect()

In [ ]:
count_q_bf = 0
sum_q_bf = 0
for index in tqdm(range(len(x_val_sample))):
    if(adv_evo_strategy_bf[index].queries > 1):
        count_q_bf +=1
        sum_q_bf += adv_evo_strategy_bf[index].queries
        
print("Average queries for EVO algo:", sum_q_bf/count_q_bf)
print("Total count of perturbed images (classified correctly initially):", perturbed_images_bf)

In [ ]:
671/len(x_val_sample)

In [ ]:
import math
from tqdm import tqdm
l2_dists = []
for index_diff in tqdm(range(50)):
    diff = np.abs(adv_evo_strategy[index_diff].get_best_candidate() - x_val_sample[index_diff])
#     diff = np.reshape(diff, (32, 32, 3))
    l2_dist = math.sqrt(np.sum(np.reshape(diff, (-1))**2))
    l2_dists.append(l2_dist)
#     print("L2 distance:", math.sqrt(np.sum(np.reshape(diff, (-1))**2)))
#     plt.imshow(np.reshape(adv_evo_strategy[index_diff].get_best_candidate(), (28, 28)))
#     plt.show()
#     print("Prediction:", model.predict(np.array([adv_evo_strategy[index_diff].get_best_candidate()])))

In [ ]:
np.shape(x_val_sample[0])

In [ ]:
np.mean(l2_dists)/(255 * 224 * 224)

In [ ]:
importlib.reload(EvoStrategy)
# TODO: add per channel perturbation, verify that it succeeds faster
# TODO: momentum approach
perturbed_images = 0
adv_evo_strategy = {}
for index in range(100):
    print()
    print(index)
    img = x_val_sample[index]
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=20, 
        one_step_perturbation_pixel_count=10
    )
    no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=50)
    if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
        perturbed_images += 1

In [ ]:
count_q = 0
sum_q = 0
for index in range(0,100):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

In [ ]:
importlib.reload(EvoStrategy)
import gc
# TODO: add per channel perturbation, verify that it succeeds faster
# TODO: momentum approach
perturbed_images = 0
adv_evo_strategy = {}
for index in range(100):
    print()
    print(index)
    img = x_val_sample[index]
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=50, 
        one_step_perturbation_pixel_count=10
    )
    no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=40)
    if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
        perturbed_images += 1
    gc.collect()

In [ ]:
count_q = 0
sum_q = 0
for index in range(0,100):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

In [ ]:
print("HERE")

In [ ]:
importlib.reload(SimbaWrapper)
simba_wrapper = SimbaWrapper.SimbaWrapper(model, x_val_sample, y_val_one_hot_sample, 0.1, max_queries=2000, max_l0_distance=150)

In [ ]:
simba_wrapper.run_simba()

In [ ]:
simba_wrapper.X_modified[0].min()

In [ ]:
x_val_sample[0].min()